In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('./data/gapminder.tsv', sep='\t')
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [3]:
# groupby() 함수를 사용해서 특정 열을 기준으로 데이터를 그룹화할 수 있다.
age_lifeExp_by_year = df.groupby('year').lifeExp.mean()
print(type(age_lifeExp_by_year))
age_lifeExp_by_year

<class 'pandas.core.series.Series'>


year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [4]:
age_lifeExp_by_year = df.groupby(['continent', 'year']).lifeExp.mean()
age_lifeExp_by_year

continent  year
Africa     1952    39.135500
           1957    41.266346
           1962    43.319442
           1967    45.334538
           1972    47.450942
           1977    49.580423
           1982    51.592865
           1987    53.344788
           1992    53.629577
           1997    53.598269
           2002    53.325231
           2007    54.806038
Americas   1952    53.279840
           1957    55.960280
           1962    58.398760
           1967    60.410920
           1972    62.394920
           1977    64.391560
           1982    66.228840
           1987    68.090720
           1992    69.568360
           1997    71.150480
           2002    72.422040
           2007    73.608120
Asia       1952    46.314394
           1957    49.318544
           1962    51.563223
           1967    54.663640
           1972    57.319269
           1977    59.610556
           1982    62.617939
           1987    64.851182
           1992    66.537212
           1997    68.02051

groupby() 함수와 같이 사용하는 집계 함수

In [5]:
# count() => 누락값을 제외한 데이터 개수
# size() => 누락값을 포함한 데이터 개수
# mean() => 평균
# std() => 표준편차
# min() => 최소값
# quantile(q = 0.25) => 1사분위수
# quantile(q = 0.50) => 중위수, 2사분위수
# quantile(q = 0.75) => 3사분위수
# max() => 최대값
# sum() => 합계
# var() => 분산
# sem() => 평균의 표준편차
# describe() => 데이터 개수, 평균, 표준편차, 최소값, 사분위수, 최대값을 모두 반환
# first() => 첫 번째 행 반환
# last() => 마지막 행 반환
# nth() => n번째 행 반환

사용자 정의 함수는 groupby() 함수의 집계 함수처럼 일괄 적용시킬 수 없기 때문에 agg() 함수를 사용해서 일괄 적용시킨다.

In [6]:
# 숫자가 저장된 리스트를 인수로 넘겨받아 평균을 계산해서 리턴하는 함수 => mean() 함수와 같은 기능이 실행된다.
def my_mean(values):
    n = len(values)
    total = 0
    for value in values:
        total += value
    return total / n

In [7]:
print(my_mean([1, 2, 3, 4, 5]))

3.0


In [8]:
# agg() 함수의 인수로 사용자 정의 함수를 () 없이 넘겨주면 실행된다.
# age_my_mean = df.groupby('year').lifeExp.my_mean() # 에러
# agg() 함수는 agg() 함수를 실행한 그룹 데이터가 자동으로 agg() 함수의 인수로 지정한 사용자 정의 함수의 인수로 넘어간다.
age_my_mean = df.groupby('year').lifeExp.agg(my_mean)
age_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [9]:
# year 별로 계산된 그룹 평균과 전체 평균의 편차를 계산하는 함수
def my_mean_diff(values, diff_value):
    n = len(values)
    total = 0
    for value in values:
        total += value
    mean = total / n
    # 그룹별 평균에서 전체 평균을 뺀 값을 리턴한다.
    return mean - diff_value

In [10]:
# 전체 평균을 계산한다.
global_mean = df.lifeExp.mean()
print(global_mean)

59.474439366197174


In [11]:
# agg() 함수가 첫 번째 인수로 그룹 데이터를 함수로 전달하고 두 번째 인수로 전체 평균을 전달한다.
age_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, global_mean)
age_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

그룹화한 1개의 열에 넘파이 집계 함수를 2개 이상 사용하기

In [12]:
# agg() 함수의 인수로 넘파이 집계 함수를 리스트에 담아 전달한다.
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142.0,49.057620,12.225956
1957,142.0,51.507401,12.231286
1962,142.0,53.609249,12.097245
1967,142.0,55.678290,11.718858
1972,142.0,57.647386,11.381953
1977,142.0,59.570157,11.227229
1982,142.0,61.533197,10.770618
1987,142.0,63.212613,10.556285
1992,142.0,64.160338,11.227380


그룹화한 2개 이상의 열에 넘파이 집계 함수를 2개 이상 사용하기

In [13]:
# 딕셔너리에 계산에 사용할 열을 key로 사용할 함수를 value로 묶어서 전달한다.
gdf = df.groupby('year').agg({'lifeExp': 'mean', 'pop': 'median', 'gdpPercap': 'sum'})
gdf

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,5.289892e+05
1957,51.507401,4282942.0,6.105160e+05
1962,53.609249,4686039.5,6.710654e+05
1967,55.678290,5170175.5,7.786787e+05
1972,57.647386,5877996.5,9.613518e+05
1977,59.570157,6404036.5,1.038470e+06
1982,61.533197,7007320.0,1.067684e+06
1987,63.212613,7774861.5,1.121931e+06
1992,64.160338,8688686.5,1.158522e+06


표준점수 계산하기  
데이터이 평균과 표준편차의 차이를 표준점수라 부른다.  
표준점수의 표준편차를 계산하면 변환된 데이터의 평균이 0이 되고 표준편차가 1이된다. 그러면 데이터가 표준화되어 서로 다른 데이터를 쉽게 비교할 수 있다.

In [14]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [15]:
# agg() 함수는 그룹별 대표값을 만들지만 transform() 함수는 그룹별 계산을 통해 데이터프레임 자체를 변환시킨다.
df['transform_z'] = df.groupby('year').lifeExp.transform(my_zscore)
df

,country,continent,year,lifeExp,pop,gdpPercap,transform_z
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,-1.656854
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,-1.731249
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,-1.786543
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,-1.848157
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,-1.894173
...,...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306,-0.081621
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786,-0.336974
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960,-1.574962
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623,-2.093346


누락값을 평균값으로 처리하기

In [16]:
# seaborn 라이브러리의 tips 데이터 셋에서 무작위로 10개의 행 데이터만 가져온 다음 total_bill 열의 값 4개를 임의로 선택해서 
# 누락값으로 바꾼다.
# 매번 같은 배열의 데이터를 추출하고 싶다면 seed() 함수를 사용해서 발생되는 난수를 고정하면 된다.
np.random.seed(42)
# pandas의 sample() 함수는 인수로 지정된 개수 만큼 전체 데이터에서 랜덤하게 데이터를 추출한다.
tips_10 = sns.load_dataset('tips').sample(10)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [17]:
# numpy의 shuffle() 함수와 permutation() 함수는 무작위로 배열을 섞는다.
# 차이점은 permutation() 함수는 섞인 배열을 리턴하기 때문에 원본이 그대로 유지되지만 shuffle() 함수는 원본 자체를 수정한다.
'''
x = np.arange(10)
print(x) # [0 1 2 3 4 5 6 7 8 9]이 출력된다. => 원본
print(np.random.permutation(x)) # 섞인 결과가 출력된다.
print('permutation():', x) # [0 1 2 3 4 5 6 7 8 9]이 출력된다. => permutation() 함수는 원본을 변형시키지 않는다.
print(np.random.shuffle(x)) # None => 원본 자체가 섞여버린다.
print('shuffle():', x) # 섞인 결곽 출력된다.
'''
pass

In [18]:
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [19]:
# total_bill 열의 누락값을 total_bill 열의 전체 평균으로 채우기
# tips_10.total_bill.fillna(tips_10.total_bill.mean())

In [20]:
# total_bill 열의 누락값을 단순히 total_bill 열의 전체 평균으로 채우면 안된다.
# 현재 tips_10의 데이터는 여성보다 남성이 더 많기 때문에 여성과 남성을 구분해서 total_bill 열의 평균값을 구하지 않으면
# 여성 데이터가 남성 데이터의 영향(간섭)을 받아서 여성 데이터가 훼손될 수 있다.
# 성별별로 그룹화 해 보면 남성의 누락값은 3개, 여성의 누락값은 1개인 것을 알 수 있다.
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [21]:
print('total_bill 전체 평균:', tips_10.total_bill.mean())
print(tips_10.groupby('sex').total_bill.mean())

total_bill 전체 평균: 16.611666666666668
sex
Male      17.9525
Female    13.9300
Name: total_bill, dtype: float64


In [22]:
# 성별을 구분해서 total_bill 열의 데이터를 받아 평균을 계산해서 누락값을 수정하는 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [23]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
total_bill_group_mean

24     19.8200
6       8.7700
153    17.9525
211    17.9525
198    13.9300
176    17.9525
192    28.4400
124    12.4800
9      14.7800
101    15.3800
Name: total_bill, dtype: float64